## Practica 1 - Computer Vision

In [ ]:
import os
import gdown

# Crear el directorio de datos si no existe
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# URL de Google Drive en formato correcto para gdown
url = "https://drive.google.com/uc?id=1iGBv-VT5mm1RiouD-U2qWcU3BYqp2OwE"
zip_filename = "practica_1_dataset.zip"
zip_path = os.path.join(data_dir, zip_filename)

# Descargar el archivo
gdown.download(url, zip_path, quiet=False)

In [199]:
import os
from zipfile import ZipFile
from concurrent.futures import ThreadPoolExecutor

data_dir = "data"
zip_filename = "practica_1_dataset.zip"
zip_path = os.path.join(data_dir, zip_filename)
if not os.path.exists(zip_path.split(".")[0]):
    with ZipFile(zip_path, 'r') as zf:
        with ThreadPoolExecutor() as exe:
            for file in zf.namelist():
                if not file.startswith("__MACOSX"):
                    exe.submit(zf.extract, file, path=data_dir)

In [200]:
import tensorflow as tf

ds_files = tf.data.Dataset.list_files("data/test/" + '*.jpg', shuffle=False)
print("Total image files:", len(ds_files))
print("Filenames shape:", ds_files.element_spec)

example = next(ds_files.take(1).as_numpy_iterator())
example

Total image files: 63
Filenames shape: TensorSpec(shape=(), dtype=tf.string, name=None)


b'data/test/IMG_2289_jpeg_jpg.rf.fe2a7a149e7b11f2313f5a7b30386e85.jpg'

In [201]:
import pandas as pd
df = pd.read_csv("data/test/annotations.csv")
count_class = df.groupby(["filename", "class"]).size()
print(f"{count_class.shape = }")
count_class.head()

count_class.shape = (83,)


filename                                                   class  
IMG_2289_jpeg_jpg.rf.fe2a7a149e7b11f2313f5a7b30386e85.jpg  puffin      1
IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg  penguin    23
IMG_2319_jpeg_jpg.rf.6e20bf97d17b74a8948aa48776c40454.jpg  penguin     8
IMG_2347_jpeg_jpg.rf.7c71ac4b9301eb358cd4a832844dedcb.jpg  penguin     2
IMG_2354_jpeg_jpg.rf.396e872c7fb0a95e911806986995ee7a.jpg  penguin     5
dtype: int64

In [202]:
df["area"] = (df["xmax"] - df["xmin"]) * (df["ymax"] - df["ymin"])
sum_area = df.groupby(["filename", "class"])["area"].sum()
print(f"{sum_area.shape = }")
sum_area.head()

sum_area.shape = (83,)


filename                                                   class  
IMG_2289_jpeg_jpg.rf.fe2a7a149e7b11f2313f5a7b30386e85.jpg  puffin      94864
IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg  penguin     32549
IMG_2319_jpeg_jpg.rf.6e20bf97d17b74a8948aa48776c40454.jpg  penguin     29583
IMG_2347_jpeg_jpg.rf.7c71ac4b9301eb358cd4a832844dedcb.jpg  penguin    250311
IMG_2354_jpeg_jpg.rf.396e872c7fb0a95e911806986995ee7a.jpg  penguin     14881
Name: area, dtype: int64

In [203]:
score = pd.Series(sum_area * count_class, name="score").reset_index()
print(f"{score.shape = }")
score.head()

score.shape = (83, 3)


,filename,class,score
0,IMG_2289_jpeg_jpg.rf.fe2a7a149e7b11f2313f5a7b3...,puffin,94864
1,IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125...,penguin,748627
2,IMG_2319_jpeg_jpg.rf.6e20bf97d17b74a8948aa4877...,penguin,236664
3,IMG_2347_jpeg_jpg.rf.7c71ac4b9301eb358cd4a8328...,penguin,500622
4,IMG_2354_jpeg_jpg.rf.396e872c7fb0a95e911806986...,penguin,74405


In [204]:
df_labels = score.groupby("filename").max().drop("score", axis=1)
print(f"{df_labels.shape = }")
df_labels.head()

df_labels.shape = (63, 1)


,class
filename,
IMG_2289_jpeg_jpg.rf.fe2a7a149e7b11f2313f5a7b30386e85.jpg,puffin
IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg,penguin
IMG_2319_jpeg_jpg.rf.6e20bf97d17b74a8948aa48776c40454.jpg,penguin
IMG_2347_jpeg_jpg.rf.7c71ac4b9301eb358cd4a832844dedcb.jpg,penguin
IMG_2354_jpeg_jpg.rf.396e872c7fb0a95e911806986995ee7a.jpg,penguin
